### Forest structure using PDAL + Python

Dr Adam Steer, November 2019.

In [22]:
NODATA_VALUE = -9999
LCF_HEIGHTS = [0, 0.5, 1, 2, 3]


In [23]:
#imports
import pdal
import numpy as np
import json

from shapely.geometry import Point
from shapely.geometry import MultiPolygon
from shapely.geometry import box
#from shapely.strtree import STRtree

import geopandas as gpd
import pandas as pd
import osmnx as ox

import os

import sys

sys.path.insert(0, os.path.abspath('../'))
sys.path.insert(0, os.path.abspath('../../callingelvis'))

# not using this, using geopandas instead
from rtree import index

# this is needed to create a raster from the output array
from osgeo import gdal
import osgeo.osr as osr

In [24]:
from callingelvis import anybodyhome

import forestmetrics
import forestmetrics.utils as forestutils

#import forestpipeline


In [25]:
def compute_tern_products(metadata, points, sindex, resolution, lasfile, outpath):
    """
    Wrapper to iterate over the input data and generate rasters for each product.
    
    *note this part could be paralellised - maybe per-product, or per-cell
    
    Each grid square processed in this loop corresponds to one pixel in an output raster.
    
    """
    
    #set up an 'output resolution' sized grid - like a fishnet grid.
    # each polygon in the resulting set covers an area of 'resolution X resolution'
    pixel_grid = forestutils.gen_raster_cells(metadata, resolution)
    
    #set up output rasters
    
    # get tile width and height
    tile_width = metadata["metadata"]["readers.las"]["maxx"] - metadata["metadata"]["readers.las"]["minx"]
    tile_height = metadata["metadata"]["readers.las"]["maxy"] - metadata["metadata"]["readers.las"]["miny"]

    raster_xsize = int(np.ceil(tile_width) / resolution)
    raster_ysize = int(np.ceil(tile_height) / resolution)
    
    #replicate for all products...
    vcf_raster = np.zeros((raster_xsize, raster_ysize))
    cth_raster = np.zeros((raster_xsize, raster_ysize))
    cbh_raster = np.zeros((raster_xsize, raster_ysize))
    fbf_raster = np.zeros((raster_xsize, raster_ysize))
    cli_raster = np.zeros((raster_xsize, raster_ysize))
    
    #internal loop around grid squares covering the LAS tile.
    # this is another ppoint for parallelisation - since we can set up a list of geometries
    # and cast that at multipuple processes, setting up one process per grid square
    # another way to do this would be to recast this loop block into a function which can  
    # be called by one process per product
    # the second strategy seems easier, then only one process is trying to write into each
    # output array.
    
    for pixel in pixel_grid:
        
        #compute output array index for this cell:
        poly_x, poly_y = pixel.centroid.xy
        
        poly_base_x = poly_x[0] - metadata["metadata"]["readers.las"]["minx"]
        poly_base_y = poly_y[0] - metadata["metadata"]["readers.las"]["miny"]
        
        array_x = int(np.floor((poly_base_x / (resolution)) ))
        array_y = int(np.floor((poly_base_y / (resolution)) ))
                
        #get points for this cell
        matches = forestutils.get_cell_points(pixel, points, sindex)
        
        #compute in order
        #VCF
        vcf_raster[array_x, array_y] = forestmetrics.comp_vcf(matches)
        
        
        #LCF - need stuff about levels here...

        lcf_raster[array_x, array_y] = forestmetrics.comp_lcf(points)
        
        #CTH
        cth_raster[array_x, array_y] = forestmetrics.comp_cth(matches)
        
        #CBH
        cbh_raster[array_x, array_y] = forestmetrics.comp_cbh(matches)
        
        #FBF
        fbf_raster[array_x, array_y] = forestmetrics.comp_fbf(matches)
        
        #CLI
        cli_raster[array_x, array_y] = forestmetrics.comp_cli(matches)
        
        
    #end of computing stuff, time to make outputs...
    
    #set up GDAL parameters
    
    wktcrs = metadata["metadata"]["readers.las"]["comp_spatialreference"]
    
    raster_parameters = {}
    raster_parameters["width"] = np.shape(vcf_raster)[0]
    raster_parameters["height"] = np.shape(vcf_raster)[1]
    raster_parameters["upperleft_x"] = metadata["metadata"]["readers.las"]["minx"]
    raster_parameters["upperleft_y"] = metadata["metadata"]["readers.las"]["maxy"]
    raster_parameters["resolution"] = resolution
    raster_parameters["projection"] = wktcrs
    
    if (not os.path.isdir(outpath + "/vcf")):
        os.mkdir(outpath + "/vcf")
    if (not os.path.isdir(outpath + "/cth")):
        os.mkdir(outpath + "/cth")
    if (not os.path.isdir(outpath + "/cbh")):
        os.mkdir(outpath + "/cbh")
    if (not os.path.isdir(outpath + "/fbf")):
        os.mkdir(outpath + "/fbf")
    if (not os.path.isdir(outpath + "/cli")):
        os.mkdir(outpath + "/cli")
        
    fileroot = forestutils.make_file_rootname(lasfile)

    #set output filenames
    vcf_raster_path = outpath + "/vcf/" + fileroot + "-VCF-" + str(resolution) + "m.tiff"
    cth_raster_path = outpath + "/cth/" + fileroot + "-CTH-" + str(resolution) + "m.tiff"
    cbh_raster_path = outpath + "/cbh/" + fileroot + "-CBH-" + str(resolution) + "m.tiff"
    fbf_raster_path = outpath + "/fbf/" + fileroot + "-FBF-" + str(resolution) + "m.tiff"
    cli_raster_path = outpath + "/cli/" + fileroot + "-CLI-" + str(resolution) + "m.tiff"


    #write geotiffs and return arrays for inspection...

    forestutils.write_product_geotiff(vcf_raster, vcf_raster_path, raster_parameters)
    forestutils.write_product_geotiff(cth_raster, cth_raster_path, raster_parameters)
    forestutils.write_product_geotiff(cbh_raster, cbh_raster_path, raster_parameters)
    forestutils.write_product_geotiff(fbf_raster, fbf_raster_path, raster_parameters)
    forestutils.write_product_geotiff(cli_raster, cli_raster_path, raster_parameters)
    
    tern_products = {}
    tern_products["vcf"] = vcf_raster
    tern_products["cth"] = cth_raster
    tern_products["cbh"] = cbh_raster
    tern_products["fbf"] = fbf_raster
    tern_products["cli"] = cli_raster
    
    return(tern_products)

## Testing functionality using a local file
The following section generates metrics from a local LAZ file. Plugging in download mechanics from ELVIS will be added later

In [26]:
#lidar test file - Mt Ainslie, chosen for varied vegetation cover and topography
# this is pretty big, try it out if you've got more resources than my macbook pro!

# thinking ahead, there will probably end up being a file splitting pre-process for 
# tiles like these... capping at say, 20 mill points. Sorting data before splitting
# will be essential.

#lasfile = "/Volumes/Antares/ACT-lidar/8ppm/callingelvis-testdata/ACT2015_8ppm-C3-AHD_6966094_55.laz"

In [27]:
#lasfile = "/Volumes/Antares/fire-test/NSW Government - Spatial Services-2/Point Clouds/AHD/StAlbans201709-LID2-C3-AHD_2866308_56_0002_0002/StAlbans201709-LID2-C3-AHD_2866308_56_0002_0002.las"

In [28]:
lasfile = "../../callingelvis-sampledata/Berridale201802-LID2-C3-AHD_6585974_55_0002_0002.las"

In [29]:
# dump everything from memory
points = None
df = None
vcf_raster = None
cth_raster = None
fbf_raster = None

In [30]:
metadata = forestutils.readlasmetadata(lasfile)

In [31]:
%%time
# this part of the process is simply reading from the source file. No analysis yet.

points = forestutils.readlasfile(lasfile)

CPU times: user 7.9 s, sys: 174 ms, total: 8.08 s
Wall time: 8.07 s


In [32]:
# uncomment to examine LAS metadata
#metadata

In [39]:
%%time

#here we read points into a GeoDataFrame and dump the labelled array.
# this is a pretty expensive step RAM wise, we're duplicating all the points...

df = forestutils.pdal2df(points)

# set the points structured array to None, it isn't used anymore
points = None

CPU times: user 1min 34s, sys: 1.02 s, total: 1min 35s
Wall time: 1min 36s


In [40]:
%%time

# here we generate an RTree index on the dataframe using GeoPandas.
# also pretty expensive... 

sindex = forestutils.spatialindex(df)

CPU times: user 1min 55s, sys: 1.16 s, total: 1min 56s
Wall time: 1min 57s


In [41]:
# set an output resolution

resolution = 25

In [42]:
metadata["metadata"]["readers.las"]["minx"]

658000

In [37]:
%%time

#produce some rasters!

tern_products = compute_tern_products(metadata, df, sindex, resolution, lasfile, "../../")

NameError: name 'sindex' is not defined

In [20]:
from matplotlib import pyplot as plt

%matplotlib inline

In [21]:
plt.imshow(np.rot90(tern_products["vcf"]))
plt.colorbar()
plt.title("Vegetation cover fraction (VCF)")

NameError: name 'tern_products' is not defined

In [ ]:
plt.imshow(np.rot90(tern_products["cth"]), vmin = 0)
plt.colorbar()
plt.title("Canopy top height (CTH)")


In [ ]:
plt.imshow(np.rot90(tern_products["cbh"]), vmin = 0)
plt.colorbar()
plt.title("Canopy base height (CBH)")

In [ ]:
plt.imshow(np.rot90(tern_products["cth"] -tern_products["cbh"]))
plt.colorbar()
plt.title("Canopy top and base height difference")

In [ ]:
plt.imshow(np.rot90(tern_products["fbf"]))
plt.colorbar()
plt.title("Building fraction (FBF)")

In [ ]:
plt.imshow(np.rot90(tern_products["cli"]))
plt.colorbar()
plt.title("Canopy Layering Index (CLI)")

In [ ]:
polygons = gen_raster_cells(metadata, resolution)

In [ ]:
polygons[40]

In [ ]:
matches = get_cell_points(polygons[55], df, sindex)

In [ ]:
comp_cli(matches)

In [ ]:
comp_cli(matches)

In [ ]:
np.divide(854.0, 284.6666666666667)

In [ ]:
cli_alt(matches)

In [ ]:
comp_density(matches, resolution)

## code purgatory

In [ ]:

# an attempt at building an index without pandas. Pandas was far easier
"""
%%time


## rtree index building straight from the point dataset... which also duplicates the point set...

idx = index.Index()
for pid, point in enumerate(points[0]):
    idx.insert(pid, (point[0], point[1],point[0], point[1]), point)
"""